<a href="https://colab.research.google.com/github/martinacaramaschi/TPT-PE-thematic-analysis/blob/main/03TPT_text_improver_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPT articles text improver
*Update: 20 th November 2024*

## Overview
This notebook aims to improve the text of artilce by removing unneeded text before the title.


# Importing datasets


In [ ]:
# importing TPT table containg cleaned articles
import pickle

directory_name = '/content/drive/MyDrive/Colab Notebooks/TPT_PE_review/'
document_name = 'TPT_metadata_cleaned_fulltext_cleaned_v2.pkl'

with open(directory_name + document_name, 'rb') as f:
    tpt_data = pickle.load(f)

In [ ]:
# importing PE table containg cleaned articles
directory_name = '/content/drive/MyDrive/Colab Notebooks/TPT_PE_review/'
document_name = 'physics_education_metadata_with_fulltext_cleaned_v2.pkl'

with open(directory_name + document_name, 'rb') as f:
    pe_data = pickle.load(f)

Visualising TPT_data and PE_data

In [ ]:
tpt_data.info()

# TPT 7203 entries / 35 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7203 entries, 0 to 7202
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   index                   7203 non-null   int64 
 1   filename                7203 non-null   object
 2   year                    7203 non-null   int64 
 3   title                   7203 non-null   object
 4   author_list             7203 non-null   object
 5   volume                  7203 non-null   object
 6   issue                   7203 non-null   object
 7   processed_len           7203 non-null   int64 
 8   page                    7203 non-null   object
 9   page_len                7203 non-null   int64 
 10  overlap                 7203 non-null   int64 
 11  pdf2fix                 7203 non-null   int64 
 12  pdf_pages               7203 non-null   int64 
 13  overlapnext             7203 non-null   int64 
 14  overlapprev             7203 non-null   int64 
 15  URL 

In [ ]:
pe_data.info()
# PE 6445 entries / 12 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6445 entries, 0 to 6444
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             6445 non-null   object
 1   authors           6445 non-null   object
 2   publication_year  6445 non-null   object
 3   doi               6445 non-null   object
 4   volume            6445 non-null   object
 5   issue             6445 non-null   object
 6   fpage             6445 non-null   object
 7   lpage             6445 non-null   object
 8   pdf_filename      6445 non-null   object
 9   zip_filename      6445 non-null   object
 10  fulltext          6445 non-null   object
 11  word_count        6445 non-null   int64 
dtypes: int64(1), object(11)
memory usage: 604.3+ KB


# REMOVING TEXT BEFORE TITLE & AFTER REFERENCES

In [ ]:
# defining the function to remove text before
def remove_text_beforetitle(row):
  old_text = row['fulltext']
  old_title = row['title']

  if old_title.startswith("notes: "):
    old_title = old_title[len("notes: "):]

  if old_title.startswith("Notes: "):
    old_title = old_title[len("Notes: "):]

  if old_title.startswith("NOTES: "):
    old_title = old_title[len("NOTES: "):]

  # Bringing the first part of the title
  # cause it is sufficient to find only its initial part into the text
  new_length = int(len(old_title) * (2/3))
  old_title = old_title[:new_length]

  # Find the starting index of old_title within old_text
  start_index = old_text.lower().find(old_title.lower())

  flag_before = 0

  if start_index != -1:
    # Extract the part of old_text starting from start_index
    new_text = old_text[start_index:]
    flag_before += 1
  else:
    new_text = old_text
  return pd.Series({'extracted_text': new_text, 'flag_before': flag_before})

In [ ]:
#TPT
import pandas as pd # Importing pandas to use pd.Series

# Apply the function to each row in tpt_data
tpt_data[['extracted_text','flag_before']] = tpt_data.apply(remove_text_beforetitle, axis=1)

# Now tpt_data will have a new column 'extracted_text' containing the extracted text
print(tpt_data[['fulltext','cleaned_fulltext', 'extracted_text', 'flag_before']].head())

                                            fulltext  \
0  VOLUME I -NUMBER 1 \nT H E \nAPRIL 1963 \n\nPH...   
1  International Agreements on the Measurement \n...   
2  Weight and Weightlessness \nA definition of 11...   
3  Teaching the Dynamics of Uniform Circular Moti...   
4  An Historic Occasion \nThe establishment of Th...   

                                    cleaned_fulltext  \
0  VOLUME I -NUMBER 1 \nT H E \nAPRIL 1963 \n\nPH...   
1  International Agreements on the Measurement \n...   
2  Weight and Weightlessness \nA definition of 11...   
3  Teaching the Dynamics of Uniform Circular Moti...   
4  An Historic Occasion \nThe establishment of Th...   

                                      extracted_text  flag_before  
0  New Unified Scale for Atomic Masses and Weight...            1  
1  International Agreements on the Measurement \n...            1  
2  Weight and Weightlessness \nA definition of 11...            1  
3  Teaching the Dynamics of Uniform Circular Moti...  

Showing previous and cleaned text

In [ ]:
tpt_data[['fulltext','cleaned_fulltext', 'extracted_text', 'flag_before']].sample(8)

,fulltext,cleaned_fulltext,extracted_text,flag_before
2747,Educational Materials and Equipment Company: C...,Educational Materials and Equipment Company: C...,Educational Materials and Equipment Company: C...,1
2890,"Center of Mass of a ""State"" \nBill Reid \nJack...",,"Center of Mass of a ""State"" \nBill Reid \nJack...",1
1503,An unexpected result is obtained when the lens...,An unexpected result is obtained when the lens...,Energy in the atmosphere \nKenneth L. Laws \n\...,1
6667,Significance of a Recurring Function in Energy...,Significance of a Recurring Function in Energy...,Significance of a Recurring Function in Energy...,1
6423,Treb-Bot: Development and Use of a Trebuchet S...,Treb-Bot: Development and Use of a Trebuchet S...,Treb-Bot: Development and Use of a Trebuchet S...,1
7200,"\nJochen Kuhn and Patrik Vogt, Column Editors ...","\nJochen Kuhn and Patrik Vogt, Column Editors ...",Learning the lens equation using water and sma...,1
2122,APPARATUS \nfor Teaching Physics \nRichard B. ...,APPARATUS \nfor Teaching Physics \nRichard B. ...,APPARATUS \nfor Teaching Physics \nRichard B. ...,0
6498,AstroNotes \nMotivating introductory physics s...,AstroNotes \nMotivating introductory physics s...,Motivating introductory physics students using...,1


In [ ]:
sum_of_flag = tpt_data['flag_before'].sum()
print("Sum of 'flag_before':", sum_of_flag)


Sum of 'flag_before': 5221


In [ ]:
#PE
import pandas as pd # Importing pandas to use pd.Series

# Apply the function to each row in tpt_data
pe_data[['extracted_text','flag_before']] = pe_data.apply(remove_text_beforetitle, axis=1)

# Now tpt_data will have a new column 'extracted_text' containing the extracted text
print(pe_data[['fulltext', 'extracted_text', 'flag_before']].head())

                                            fulltext  \
0  www.iop.org/journals/physed \nThe polar mesosp...   
1  Phys Educ Val 16.1981 Printed In GreatBrltaln ...   
2  and Howson (1974) suggest we need to reconcile...   
3  \nNEW MATERIALS \nMaterials, bonding mechanism...   
4  The freewheeling cyclist \nWim Hennekam and Mi...   

                                      extracted_text  flag_before  
0  The polar mesosphere \nRay Morris and Damian M...            1  
1  Matter for illumination \n\nJoan L Hawes \nWho...            1  
2  and Howson (1974) suggest we need to reconcile...            0  
3  Materials, bonding mechanisms and physical pro...            1  
4  The freewheeling cyclist \nWim Hennekam and Mi...            0  


In [ ]:
sum_of_flag = pe_data['flag_before'].sum()
print("Sum of 'flag_before':", sum_of_flag)

Sum of 'flag_before': 5002


In [ ]:
tpt_data.info()
pe_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7203 entries, 0 to 7202
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   index                   7203 non-null   int64 
 1   filename                7203 non-null   object
 2   year                    7203 non-null   int64 
 3   title                   7203 non-null   object
 4   author_list             7203 non-null   object
 5   volume                  7203 non-null   object
 6   issue                   7203 non-null   object
 7   processed_len           7203 non-null   int64 
 8   page                    7203 non-null   object
 9   page_len                7203 non-null   int64 
 10  overlap                 7203 non-null   int64 
 11  pdf2fix                 7203 non-null   int64 
 12  pdf_pages               7203 non-null   int64 
 13  overlapnext             7203 non-null   int64 
 14  overlapprev             7203 non-null   int64 
 15  URL 

## Saving final datasets

In [ ]:
# Save the cleaned DataFrame as a pickle file
with open(directory_name + 'physics_education_metadata_final.pkl', 'wb') as f:
    pickle.dump(pe_data, f)

with open(directory_name + 'TPT_metadata_final.pkl', 'wb') as f:
    pickle.dump(tpt_data, f)